In [1]:
import setup_django
setup_django.init()

In [3]:
import numpy as np
import pandas as pd
from django.contrib.auth import get_user_model
from django.db.models import F, Count, Sum, IntegerField
from playlists.models import MovieProxy
from ratings.models import Rating
import random

User = get_user_model()

In [22]:
rating_obj = Rating.objects.filter(user__username='admin', value__gte=5).first()
rating_obj.content_object

<MovieProxy: The Million Dollar Hotel (2000)>

In [23]:
movie_id = rating_obj.content_object.id
value = rating_obj.value
user_id = rating_obj.user_id

In [24]:
print(f"Movie {movie_id} was rated {value} by {user_id}")

Movie 318 was rated 5 by 1


In [25]:
# Other users id by the same value

other_user_ids = Rating.objects.filter(
        object_id=rating_obj.object_id, 
        content_type=rating_obj.content_type, 
        value__gte=rating_obj.value
).exclude(user=rating_obj.user).values_list('user_id', flat=True)
#print(other_user_ids)

# Other user ratings greater then 4
highly_rated = Rating.objects.filter(user_id__in=other_user_ids, value__gte=4)

In [26]:
rec_users = []
rec_movies = []
for rating_obj in highly_rated:
    if rating_obj.user not in rec_users:
        rec_users.append(rating_obj.user)
    object_id = rating_obj.object_id
    if object_id not in rec_movies:
        rec_movies.append(object_id)

In [27]:
print(len(rec_users), len(set(rec_movies)))

207 4795


In [29]:
# End up beeing a lot of movies!!
# Valid for a much smaller dataset

In [30]:
import numpy as np

In [31]:
# Write a list of 2 movies attributes
movie_1 = ['sci-fi', 'comedy', 'Stars latest action movie star']
movie_2 = ['historical fiction', 'drama', 'Stars latest action movie star']

# Write a list of a User's preferences
user_a = ['comedy', 'sci-fi', 'Stars latest action movie star']
print(f"User has {len(user_a)} primary preferences")

# Total number of potential features
num_unique_features = len(set(movie_1 + movie_2))
print(f"These movies have {num_unique_features} unique features.")

User has 3 primary preferences
These movies have 5 unique features.


In [32]:
my_preferences = np.array([.98, .85, .75])

In [33]:
movie_attribute_scores = np.array([.96, .99, .92]) 

movie_2_attribute_scores = np.array([-.93, -.45, .5])

In [34]:
(my_preferences * movie_attribute_scores).sum() 

np.float64(2.4723)

In [35]:
(my_preferences * movie_2_attribute_scores).sum()

np.float64(-0.9189)

In [36]:
max_score = (np.array([1,1,1]) * np.array([1,1,1])).sum()
min_score = (np.array([1,1,1]) * np.array([-1,-1,-1])).sum()
print(max_score, min_score)

3 -3


In [38]:
ids = MovieProxy.objects.all().popular()[:15]
most_active_users = Rating.objects.all().annotate(
    user_count=Count("user")
).order_by('-user_count').values_list('user_id')[:5_000]

ratings_qs =  Rating.objects.filter(
    object_id__in=ids, 
    user_id__in=most_active_users, 
    active=True, value__gte=0
).annotate(movie=F("object_id")).values('user', 'value', 'movie')

df = pd.DataFrame(ratings_qs)

In [39]:
df.pivot_table(index='user', columns=['movie'], values='value', fill_value='')[:15]

movie,1,50,260,296,318,480,527,589,593,608,1196,1198,1210,1270,2571
user,,,,,,,,,,,,,,,
1,,,,,5.0,,,,,,,,,,
2,,4.0,,4.0,,4.0,4.0,5.0,3.0,,,,,,
3,,,,5.0,5.0,,3.0,,3.0,,,,3.0,,
4,,,5.0,5.0,,5.0,,5.0,,,5.0,5.0,5.0,5.0,
6,,,,,,,,,,,,,,,1.0
7,3.0,,5.0,,5.0,4.0,,3.0,,,5.0,5.0,5.0,3.0,
8,,5.0,4.0,4.0,5.0,,5.0,4.0,5.0,,4.0,4.0,4.0,4.0,5.0
9,4.0,,,,4.0,,5.0,,4.0,5.0,,,,,5.0
10,,5.0,,,4.0,,,,,,4.0,4.0,4.0,,5.0


In [42]:
movie_1 = np.array([1, 1, 1, 1, 1])
user_1 = np.array([1, 1, 1, 1, 1])
(user_1 * movie_1).sum()


np.int64(5)